
**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Edición 2021**


---

# Trabajo práctico entregable - Parte 2


* Nadia Finzi
* Roxana Noelia Villafañe
* María Roberta Devesa
* Ivana Feldfeber
* Ezequiel Kinigsberg

In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn

seaborn.set_context('talk')

## Lectura del dataset

En la notebook 00 se explican los detalles de la siguiente sección.

In [ ]:
url = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/sysarmy_survey_2020_processed.csv'
df = pd.read_csv(url)

In [ ]:
df[:3]

,profile_gender,profile_age,work_country,work_province,profile_years_experience,work_years_in_company,work_years_in_current_position,work_people_in_charge_of,profile_studies_level,profile_studies_level_state,profile_career,profile_university,profile_specialization_courses,profile_open_source_contributions,profile_programming_as_hobby,work_role,tools_platform,tools_programming_languages,tools_frameworks,tools_data_bases,tools_qa_testing,tools_IDEs,tools_work_pc_os,tools_cellphone_os,work_on_call_duty,salary_on_call_duty_charge,work_on_call_duty_charge_type,work_contract_type,salary_monthly_BRUTO,salary_monthly_NETO,salary_in_usd,salary_satisfaction,salary_comparison_last_semester,salary_has_bonus,salary_bonus_tied_to,salary_inflation_adjustment_2020,salary_percentage_inflation_adjustment_2020,salary_month_last_inflation_adjustment,work_has_violence_situations,profile_sexual_orientation,profile_has_disabilities,profile_has_disabilities_hiring_difficulties,company_employee_number,company_main_activity,company_recommended,company_diversity_policies,company_extra_benefits,company_best_companies_city
0,Mujer,26,Argentina,Ciudad Autónoma de Buenos Aires,3.0,3.0,3.0,0,Universitario,En curso,Ingeniería en Sistemas de Información,UTN - Universidad Tecnológica Nacional,"Sí, los pagó un empleador",No,Sí,Consultant,Ninguna,Ninguno,Ninguno,Ninguna,Ninguna,Ninguno,GNU/Linux,Android,No,0.0,Bruto,Full-Time,53000.0,43000.0,NaN,1,1,No,No recibo bono,No,0.0,0,En mi trabajo actual,Homosexual,NaN,NaN,501-1000,Servicios / Consultoría de Software / Digital,7,2,"Capacitaciones y/o cursos, Comidas pagas / sub...",NaN
1,Hombre,29,Argentina,Corrientes,5.0,2.0,2.0,4,Universitario,En curso,Ingeniería en Sistemas de Información,UTN - Universidad Tecnológica Nacional,"Sí, de forma particular, Sí, los pagó un emple...",No,Sí,Developer,"Docker, Google Cloud Platform/App Engine","HTML, Javascript, Python","Django, jQuery","Informix, MySQL, PostgreSQL",Ninguna,"IPython / Jupyter, Sublime Text, Visual Studio...",GNU/Linux,Android,No,0.0,Bruto,Full-Time,79000.0,63000.0,NaN,2,2,Menos de un sueldo,Performance de la compañía,Uno,10.0,1,Jamás,Heterosexual,Visual,No,201-500,Otras industrias,8,9,"Horarios flexibles, Stock options / RSUs, Viát...",NaN
2,Mujer,22,Argentina,Ciudad Autónoma de Buenos Aires,2.0,0.0,0.0,0,Secundario,Completado,NaN,NaN,"Sí, de forma particular, Sí, los pagó un emple...",No,Sí,Infosec,"Linux, VMWare",Ninguno,Ninguno,Ninguna,Ninguna,Ninguno,Windows,iOS,No,0.0,Porcentaje de mi sueldo,Full-Time,160000.0,127000.0,NaN,3,3,Un sueldo,Performance individual,No,0.0,0,En un trabajo anterior,Bisexual o queer,NaN,No,2001-5000,Otras industrias,6,9,"Clases de gimnasia online, Comidas pagas / sub...",NaN


In [ ]:
df[['profile_gender', 'salary_monthly_NETO']].groupby('profile_gender').describe()

salary_monthly_NETO                ...                     
                             count          mean  ...       75%        max
profile_gender                                    ...                     
Hombre                      4944.0  98836.063558  ...  110000.0  2080000.0
Mujer                        908.0  73501.414465  ...   92000.0   800000.0
Otros                         31.0  95441.354839  ...  117500.0   380000.0

[3 rows x 8 columns]

In [ ]:
def medidas_estadisticas(grupo):
  return grupo.count(), grupo.mean(), grupo.var()

In [ ]:
is_man = df.profile_gender == 'Hombre'

groupA = df[(df.salary_monthly_NETO > 1000) & is_man & (df.salary_monthly_NETO.notna())].salary_monthly_NETO
groupB = df[(df.salary_monthly_NETO > 1000) & ~is_man & (df.salary_monthly_NETO.notna())].salary_monthly_NETO

In [ ]:
(hombre_n, hombre_media, hombre_var) = medidas_estadisticas(groupA)
(otro_n, otro_media, otro_var) = medidas_estadisticas(groupB)


In [ ]:
df[(df.salary_monthly_NETO > 1000) & (df.salary_monthly_NETO.notna())]\
  [['profile_gender', 'salary_monthly_NETO']].groupby('profile_gender').describe()

salary_monthly_NETO                 ...                     
                             count           mean  ...       75%        max
profile_gender                                     ...                     
Hombre                      4815.0  101481.624343  ...  110000.0  2080000.0
Mujer                        863.0   77328.913778  ...   93000.0   800000.0
Otros                         28.0  105660.714286  ...  122750.0   380000.0

[3 rows x 8 columns]

In [ ]:
alpha = 0.05
# alpha / 2 = 0.025 

## Ejercicio 1: Estimación

**Consigna:**  Calcular una estimación puntual y un intervalo de confianza de nivel (1-alpha) para la resta entre la media del salario Neto para Hombres menos la media del salario Neto para otros géneros(diferencia de las medias entre el grupoA y grupoB). 
¿Cómo se relaciona este intervalo de confianza con el test de hipótesis?

Teniendo las siguientes fórmulas: 

$ T = \frac{(\bar{X}-\bar{Y})-(\mu_{1} - \mu_{2})}{s.\sqrt{\frac{n_1 + n_2}{n_1 . n_2}}} \sim t_{n_1+n_2-2}$

$s^2 = \frac{\sum_{1}^{n_1}(X_i - \bar{X})^2 + \sum_{1}^{n_2}(Y_i - \bar{Y})^2}{n_1+n_2-2}$

$[\bar{X} - t_{\frac{\alpha }{2},n-1} . \frac{s}{\sqrt{n}}  , \bar{X} + t_{\frac{\alpha }{2},n-1} . \frac{s}{\sqrt{n}}]$

Calculamos la diferencia entre las distribuciones:

In [ ]:
print (((hombre_media - otro_media) - 1.96 * np.sqrt(hombre_var/hombre_n + otro_var/otro_n)), " < mu1 - mu2 < ", ((hombre_media - otro_media) + 1.96 * np.sqrt(hombre_var/hombre_n + otro_var/otro_n)))

18560.5807418027  < mu1 - mu2 <  27964.16605803376


El intervalo de diferencia de medias entre el grupo A y el grupoB esta comprendido entre 18560.5807418027 y 27964.16605803376

$P(18560.58 < u_1 - u_2 < 27964.16) = 1-alfa$

Un intervalo de confianza del 95% indica que 95% de muestras de la misma población producirán intervalos de confianza que contienen el parámetro de la población.



In [ ]:
#este calculo puede resumirse mediante las siguientes lineas
import statsmodels.stats.api as sms
cm = sms.CompareMeans(sms.DescrStatsW(groupA), sms.DescrStatsW(groupB))
print (cm.tconfint_diff(usevar='unequal'))

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


(18558.016394102247, 27966.730405734153)


### Relación entre test de hipótesis e intervalo de confianza. 
* Cuando realizamos un test de hipótesis nos sirve para tomar una decisión, si rechazamos o no una hipótesis estadística en base a una muestra. El propósito es en base a los datos tomar una decisión para la investigación. 

* Sin embargo, el intervalo de confianza denota un rango dentro del cual podemos encontrar el parámetro a estimar con un determinado nivel de confianza (en este caso, 95%).  

## Ejercicio 2: Test de hipótesis



### 2.1 Formalización

Describir formalmente los distintos compenentes de un test de hipótesis para comprobar si la distribución de los salarios es distinta entre los grupos A y B.

**Hipótesis Nula**

$H_0$=0 No hay diferencia entre los valores medios de la distribución de los salarios de los grupos A y B 

**Hipótesis Alternativa**

$H_0$ distinto de 0. Hay diferencia entre los valores medios de la distribución de los salarios de los grupos A y B.

**Estadístico (Pivote)**
  * Identificar el estadístico: **El estadistico que se esta evaluando es la diferencia de medias**
  * Escribir qué distribución tiene bajo $H_0$ **Asumimos una distribución t de Student**


### 2.2 P-valor

1. Calcule el p-valor y decida si rechazar o no la hipótesis nula.
2. Interprete el resultado.

`pvalue=8.605197322133209e-22` 

Como pvalue es menor que el alfa entonces rechazamos la $H_0$

Cuanto más pequeño sea el valor-p, tanto mayor será la evidencia en contra de la hipótesis nula. 

<img src='https://www.six-sigma-material.com/images/xPvalueVisualAid.JPG.pagespeed.ic.vag9qkT_u3.jpg'>

*La imagen es esquemática del test a una cola pero sería similar a dos colas*

3. Los dos grupos de nuestra muestra tienen tamaños muy distintos. ¿Esto afecta al tests?

No, ya que estamos utilizando el test de Welch que no requiere que el tamaño de las muestras sea similar




Links útiles:
* [Test de hipótesis usando scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
* [Test de Welch](http://daniellakens.blogspot.com/2015/01/always-use-welchs-t-test-instead-of.html)

In [ ]:
import scipy.stats as st
st.ttest_ind(groupA,groupB, equal_var = False) #al poner aca False estamos haciendo un test de Welch

Ttest_indResult(statistic=9.697205976350673, pvalue=8.605197322133209e-22)

### [Opcional] 2.3 Potencia del test

Nuestra muestra, ¿era lo suficientemente grande para detectar si existe o no una diferencia entre los grupos?

1. Utilice la función `tt_ind_solve_power` para calcular el tamaño necesario de la muestra para un poder estadístico de 0.8, 0.9 y 0.95, asumiendo una significancia estadística de 0.05.
2. ¿Cómo intepretan el poder estadístico de un test? Dado su conocimiento de dominio sobre los datos, ¿les parece que esta muestra es lo suficientemente grande para ser representativo de la tendencia general? ¿y para utilizarlo en un juicio penal contra una empresa XX por una causa de discriminación?

[Documentación](https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html)

NOTA: este análisis debería hacerse ANTES de recolectar los datos.

 La potencia de una prueba de hipótesis es la probabilidad de que la prueba rechace correctamente la hipótesis nula. La potencia de una prueba de hipótesis se ve afectada por el tamaño de la muestra, la diferencia, la variabilidad de los datos y el nivel de significancia de la prueba.

Si una prueba tiene poca potencia, es posible que no se detecte un efecto y se concluya erróneamente que no existe ninguno. Si la potencia de una prueba es demasiado alta, efectos muy pequeños y posiblemente sin importancia podrían parecer significativos. 

In [ ]:
from statsmodels.stats.power import tt_ind_solve_power

In [ ]:
effect_size = (groupA.mean() - groupB.mean()) / groupB.std()
#nobs1=None  - What we want to know
alpha = 0.05
ratio = len(groupB) / len(groupA)

In [ ]:
tt_ind_solve_power(effect_size=effect_size, alpha=0.05, power=0.80, ratio=ratio)

298.3073379956589

In [ ]:
tt_ind_solve_power(effect_size=effect_size, alpha=0.05, power=0.90, ratio=ratio)

398.79610014029686

In [ ]:
tt_ind_solve_power(effect_size=effect_size, alpha=0.05, power=0.95, ratio=ratio)

492.8117730249087

**Respuesta**
* De estos cálculos surge que a medida que le pedimos más potencia al test estadístico, es necesario que tomemos una muestra mas grande. De hecho, si quisiéramos aumentar la potencia de un test estadístico, debemos aumentar el tamaño de nuestra muestra. 
* El valor calculado corresponde al tamaño de muestras para rechazar la hipótesis nula. 
* Esta muestra es lo suficientemente grande de la tendencia general, que los sueldos de hombres superan a Otros (mujeres y minorias). Con los datos que tenemos, aunque son suficientes a nivel general, no necesariamente son suficientes para una empresa en particular. Deberíamos analizar la situación para la misma.

## Ejercicio 3: Comunicación y visualización

**Consigna:** Seleccionen un resultado que les parezca relevante a partir de alguno de los ejercicios del entregable. Diseñe e implemente una comunicación en base a este mensaje.

Elija las palabras y visualización más adecuada para que la comunicación sea entendible, efectiva y se dapte a UNA de las siguientes situaciones:

1. Una sección en un artículo de difusión a presentar como parte de una organización sin fines de lucro.
No más de 1 página A4 (o dos si los gráficos son muy grandes).
  1. Ejemplo: Alguna de las secciones [Los ecosistemas de emprendimiento de América Latina y el Caribe frente al COVID-19: Impactos, necesidades y recomendaciones](https://publications.iadb.org/es/los-ecosistemas-de-emprendimiento-de-america-latina-y-el-caribe-frente-al-covid-19-impactos-necesidades-y-recomendaciones), por ejemplo la sección *2.2. Reacciones de los emprendedores*.
  2. Ejemplo: Alguna de las secciones de [The state of gender pay gap in 2021](https://www.payscale.com/data/gender-pay-gap?tk=carousel-ps-rc-job)
  3. Puntos clave: 
    1. Simpleza de los gráficos.
    2. Comunicación en lenguaje simple a personas que no son necesariamente expertos de dominio.
    2. Selección de UNA oración sobre la que se hace énfasis.
    3. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

2. Una publicación científica o reporte técnico interno. No más de una página A4:
  2. Ejemplo: La sección de resultados de [IZA DP No. 12914: The Impact of a Minimum Wage Change on the Distribution of Wages and Household Income](https://www.iza.org/publications/dp/12914/the-impact-of-a-minimum-wage-change-on-the-distribution-of-wages-and-household-income).
  2. Ejemplo: Alguna de las secciones de [Temporary reduction in daily global CO2 emissions during the COVID-19 forced confinement](https://www.nature.com/articles/s41558-020-0797-x)
  3. Puntos clave:
    3. Nivel de detalle técnico requerido. Es necesario justificar la validez del análisis.
    4. La idea presentada puede ser más compleja. Pueden asumir que la audiencia tiene conocimiento técnico y va a analizar las visualizaciones en detalle. 
    5. Pueden presentar más en detalle las limitaciones del análisis (significancia estadística, etc.)
    2. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

3. Un tweet (o post de LinkedIn) para la cuenta de su empresa consultora que hace análisis de datos. El objetivo es promocionar un análisis de datos abiertos que van a incluir en su portfolio:
  1. Ejemplo: [Comparación vacunas covid](https://twitter.com/infobeautiful/status/1381577746527236098?s=20)
  2. Ejemplo: [Tweet del BID](https://twitter.com/el_BID/status/1388508583944507396?s=20). Lo valioso de este tweet es que usaron un único número para transmitir un mensaje. Puede ser algo así, o con un gráfico muy simple.
  3. Ejemplo: [Cambio climático](https://twitter.com/UNFCCC/status/1387732156190011394?s=20) Es un muy buen ejemplo, excepto que el gráfico no se lee nada y hay que entrar a la publicación original.
  3. Ejemplo: [¿Cuánto están los programadores en las empresas?](https://www.linkedin.com/posts/denis-rothman-0b034043_tech-career-work-activity-6793861923269054464-gS6y) (No verificamos la veracidad o seriedad de la fuente).
  4. Puntos clave:
    1. Su audiencia no va a mirar la visualización por más de unos segundos, y no tiene conocimiento técnico.
  3. Tienen que incluir además una *breve* descripción de cómo obtuvieron los datos que están presentando, que no entraría en el tweet.
